In [1]:
# Map practice from the gmaps documentation
# URL https://jupyter-gmaps.readthedocs.io/en/v0.3.2/gmaps.html

# Import the dependencies.
import pandas as pd

import numpy as np
import random

# very important to include this module - this is what was causing error
import requests

# Use the citipy module to determine city based on latitude and longitude.
from citipy import citipy

# Import the datetime module from the datetime library.
from datetime import datetime

# check do we have the config.py set up correctly and linked into this notebook? 
from config import weather_api_key

# still need to install gmaps in the back end using anaconda prompt- this might be a good office hours question
# it was still waiting for a y/n answer so it had not installed yet
import gmaps
import gmaps.datasets

import requests
# Import the API key.
from config import g_key

# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)


In [2]:
# generate some data - this is an example, I will make my own 
# why does it open the map in Geneva as the default? 

# This is from the gmaps getting started documentation
# I thought if I could get THIS to work then the other heat map would work too
# Henry (TA) Helped me fix this on July 27- I still needed to enable two modules in anaconda
# jupyter nbextension enable --py gmaps
# jupyter nbextension enable --py widgetsnbextension


earthquake_df = gmaps.datasets.load_dataset_as_df('earthquakes')
earthquake_df.head()

,latitude,longitude,magnitude
0,65.193300,-149.072500,1.70
1,38.791832,-122.780830,2.10
2,38.818001,-122.792168,0.48
3,33.601667,-116.727667,0.78
4,37.378334,-118.520836,3.64


In [3]:
locations = earthquake_df[['latitude', 'longitude']]
weights = earthquake_df['magnitude']
fig = gmaps.figure()
fig.add_layer(gmaps.heatmap_layer(locations, weights=weights))
# fig

earthquake_df

,latitude,longitude,magnitude
0,65.193300,-149.072500,1.70
1,38.791832,-122.780830,2.10
2,38.818001,-122.792168,0.48
3,33.601667,-116.727667,0.78
4,37.378334,-118.520836,3.64
...,...,...,...
8599,19.331500,-155.182200,0.80
8600,41.819700,-119.622400,2.20
8601,38.771999,-122.720497,1.91
8602,34.608333,-118.641333,1.93


In [4]:
# first test my city to pick some USA cities and get the data for them 
# my_city = citipy.nearest_city(40.1, -96.2).city_name
my_x = 39.117
my_y = -94.4

my_city = citipy.nearest_city(my_x,my_y).city_name
my_country = citipy.nearest_city(my_x,my_y).country_code
print(my_city, my_country)

independence us


In [5]:
#test random

my_rand = np.random.uniform(0.1, 6.1)
my_noodle = np.cos(my_rand)
lat_increment = round(79/13 ,2)
lng_increment = round(1.848 ,2)
my_new_x = lat_increment * my_noodle

my_new_y = lng_increment * np.sin(my_rand/2)

print(my_rand,  my_new_x, my_new_y, lng_increment)
print(np.sin(my_rand))



0.5848659200691338 5.069419592866767 0.5333230792490253 1.85
0.5520875933081623


In [6]:
# import numpy as np
last_lat = 39.5
testcoord = np.cos(last_lat/90)
lat_increment = 5
lng_increment = lat_increment / np.cos(last_lat/90)

my_rand = np.random.uniform(1.57, 3.24)
    
    # scaling the lng_increment by np.cos(last_lat/90) makes it proportional to the lat at the last pin drop
my_new_x = lat_increment * np.cos(my_rand)
my_new_y = lng_increment * np.sin(my_rand)
print (1/testcoord, my_rand, my_new_x, my_new_y, np.cos(my_rand), np.sin(my_rand))

1.1046984950480294 1.9185728824005062 -1.7040415929565853 5.1928164710353055 -0.34080831859131705 0.9401328044372026


In [7]:
# make one more alternate version of the lats list - in equal increments but "reflecting off the wall"
# for the increment pick something that is not close to a multiple of whole numbers like the quotient of two primes
# I will generate 200 values on both x and y coords that you can choose to use or not use
lattice_x = []
lattice_y = []
# starting lat/lng is geographic center of CONUS - changed to Topeka KS science center
start_lat =39.03
last_lat = 39.03

start_lng = -95.72
last_lng = -95.72
# as increments use a ratio of prime numbers so the sequence will not repeat
#lat_increment = round(59/17, 2)
# lng_increment = round(67/19, 2)

lat_increment = 2.7
lng_increment = 2.8

# This is a global constant used in the entire notebook so set it here one time only
maxsize = 200

# another idea is to give the next pin a direction by using sin and cos 
# another random number generated inside the loop could move the point an uneven increment each time
for i in range(maxsize):
    # adding a directional randomizer that moves the next point around a circle with rand = 0:2*Pi
    # restrict the movement to a sector of the circle that keeps things going in mostly one quadrant
    my_rand = np.random.uniform(0.77, 3.84)
    
    # scaling the lng_increment by np.cos(last_lat/90) makes it proportional to the lat at the last pin drop
    my_new_x = lat_increment * np.cos(my_rand)
    my_new_y = lng_increment * np.sin(my_rand) / np.cos(last_lat/90) 
    
    # will this work ? it makes the next point move in a direction around a circle
    new_lat = 24 + round((last_lat + my_new_x) % 24,2)

    # new_lng = -123 + round((last_lng + my_new_y),2)
    # debug - keep the new number between the range of 0-54 we can shift it later
    # new_lng = -96 + round((last_lng + my_new_y),2)
    new_lng = last_lng + my_new_y
    
    # next thing is a transporter - when the pin drops in the Bermuda triangle, we launch it to Seattle
    # but I overshot the mark and dropped some in the ocean so pull those back 
    if ((new_lng > -76.15) & (new_lat < 35.7)):
        new_lng = new_lng - 48.2
        new_lat = new_lat + 12.25  
    elif ((new_lng > -77.75) & (new_lat < 40.77)):
        # this is another wormhole off Long Island to Bermuda - restricted airspace - corner Moriches Inlet 40.77, -72.73
        new_lng = new_lng - 58.2 
        new_lat = new_lat + (my_rand/6.24 * 9.75)
        
    if (new_lng < -124.5):
        # these are also glass walls at certain coordinates
        new_lng = new_lng + (30 * my_rand/6)
        
    if ((new_lng < -118.5) & (new_lat < 34.17)):
        # rocket from malibu to kansas city
        new_lat = new_lat + 10
        new_lng = new_lng + 30
    elif (new_lng < -124.5):
        # up to three time zones - even scotty can't do that
        new_lng = new_lng + (45 * my_rand/6)
            
    if (new_lng > -67.5):    
        new_lng = new_lng - 30

        # fix the lats manually instead of using modulo
    if (new_lat <= 24):
        new_lat = new_lat + 25
        new_lng = new_lng - 10
    elif (new_lat >= 59.75):
        new_lat = new_lat - 22.5
        new_lng = new_lng - 10

        
    # remove print after debug
    print(last_lng, my_new_y, ((last_lng + my_new_y)))
    
        
    lattice_x.append(new_lat)
    lattice_y.append(new_lng)
    
    last_lat = round(new_lat,2)
    last_lng = round(new_lng,2)
    
# the y coordinate is incrementing too much so I have to print some debug statements 
# is it the modulo % 54 that is making it crazy - not if the change is only - yes it is the modulo
# take the modulo off and see where it goes, it might go out of range
#maybe modulo is NOT the right operator to use - it gives the remainder after division

-95.72 2.349885433680337 -93.37011456631966
-93.37 2.8056436889888774 -90.56435631101112
-90.56 2.5657568530824753 -87.99424314691753
-87.99 3.015022387315401 -84.9749776126846
-84.97 2.792256835158955 -82.17774316484105
-82.18 2.868603272467508 -79.3113967275325
-79.31 1.404418426515961 -77.90558157348404
-77.91 -1.3523495376030699 -79.26234953760307
-79.26 -1.6027117454647413 -80.86271174546475
-80.86 2.8525046851897637 -78.00749531481024
-78.01 2.717082010539561 -75.29291798946045
-123.49 2.7177697617204855 -120.7722302382795
-120.77 3.1974340846546343 -117.57256591534536
-117.57 0.09509730899611116 -117.47490269100388
-117.47 2.747565210957779 -114.72243478904223
-114.72 -1.8667637182760501 -116.58676371827605
-116.59 0.9659898974478628 -115.62401010255213
-115.62 3.00613284194359 -112.61386715805641
-112.61 0.9670842363488145 -111.64291576365119
-111.64 2.99867678208733 -108.64132321791267
-108.64 -0.42217251821802443 -109.06217251821802
-109.06 1.990791468702185 -107.069208531297

In [8]:

my_lats = np.random.uniform(low=24.000, high=49.000, size=maxsize)
# US lng ranes from -124 WA to -66.0 Maine
my_lngs = np.random.uniform(low=-125.000, high=-65.000, size=maxsize)
# my_lngs
# random doesn't really give me the coverage I want but lets see the map before we decide
# lattice_y


In [9]:
# The pin drops are all banding into the same range of lats/longs so it is still not really a random sample
# some states got multiple hits and some got none at all - see pin drop map in last frame
# maybe the only way to really cover all 50 states is to do a lattice

counter = 0

# change my_lats to lattice_x for testing
for i in range(maxsize):
    print(lattice_x[i], lattice_y[i])
    
    counter +=1
# nex t - making a bunch of citypy calls   



40.78 -93.37011456631966
39.61 -90.56435631101112
41.120000000000005 -87.99424314691753
40.43 -84.9749776126846
39.24 -82.17774316484105
40.239999999999995 -79.3113967275325
37.83 -77.90558157348404
35.41 -79.26234953760307
33.12 -80.86271174546475
32.28 -78.00749531481024
45.66 -123.49291798946045
47.09 -120.7722302382795
46.69 -117.57256591534536
43.989999999999995 -117.47490269100388
42.64 -114.72243478904223
40.47 -116.58676371827605
37.9 -115.62401010255213
37.36 -112.61386715805641
34.8 -111.64291576365119
34.46 -108.64132321791267
31.79 -109.06217251821802
29.78 -107.06920853129782
28.240000000000002 -104.63452862982356
25.72 -105.6993197452183
24.96 -102.90042004846161
46.56 -104.23967922125007
47.15 -101.09724449759074
24.05 -98.04946807304258
24.09 -95.14727954568961
45.54 -96.11659101338878
43.239999999999995 -94.44887979076971
44.9 -91.95682645948469
45.97 -89.03104297888667
46.54 -85.89384833899803
47.47 -82.86724301180516
47.86 -79.66391786484513
45.16 -79.57646275881974


In [10]:
# use the new lattice_x for this test to replace my_lats
my_zip = zip(lattice_x, lattice_y)
my_new_list = list(my_zip)
len(my_new_list)
# my_new_list[:10]

200

In [11]:
# now I have a DF of exactly 10 GPS coords
new_df = pd.DataFrame(my_new_list[:maxsize],columns=['Lat', 'Lng'])
new_df.head(20)


,Lat,Lng
0,40.78,-93.370115
1,39.61,-90.564356
2,41.12,-87.994243
3,40.43,-84.974978
4,39.24,-82.177743
5,40.24,-79.311397
6,37.83,-77.905582
7,35.41,-79.262350
8,33.12,-80.862712
9,32.28,-78.007495


In [12]:
# Create a list for holding the cities.
my_cities = []
all_my_cities = []
# Identify the nearest city for each latitude and longitude combination.
# substitute lattic_x for my_lats
counter = 0 
for lat in lattice_x:
    lat_shift = 5.2
    lng_shift = 2.5
    # using the lattice_x method we will not need the SHIFT any more
    lng = my_lngs[counter]
    my_city = citipy.nearest_city(lat+lat_shift, lng+lng_shift).city_name

    # get a list of all the cities for testing to see which city had the most hits
    all_my_cities.append(my_city)

    # If the city is unique, then we will add it to the cities list.
    if my_city not in my_cities:
        my_cities.append(my_city)
    
    counter -= 1

# Print the city count to confirm sufficient count.
# len(all_my_cities)
all_my_cities

# with 200 random points we only got 32 unique places - that seems odd
# also I would like to select only cities above a certain population, not just any city with over 500 people
# am I supposed to believe that for every pin dropped in the gulf of Mexico, 'santander jimenez' is the only nearest city?
# its hard to believe - I am going to test citypy more to see if I give it the exact coords of a city what does it return
# also I am going to randomize the pin points even further by adding a small increment to the lat value
# with a lattice for both x and y we get lompoc and san quintin a lot but i think we get more variety

['mandan',
 'smiths falls',
 'hamilton',
 'grandview',
 'baker city',
 'hamilton',
 'nampa',
 'maryville',
 'shelbyville',
 'madison heights',
 'eston',
 'stettler',
 'sioux lookout',
 'havre',
 'east wenatchee bench',
 'miles city',
 'le mars',
 'scottsbluff',
 'fallon',
 'clifton',
 'trinidad',
 'collierville',
 'havelock',
 'valdosta',
 'hamilton',
 'chapais',
 'chapais',
 'ojinaga',
 'houma',
 'sioux lookout',
 'grande-riviere',
 'invermere',
 'cochrane',
 'dauphin',
 'dryden',
 'hudson bay',
 'sioux lookout',
 'kenora',
 'abbotsford',
 'hornepayne',
 'helena',
 'matane',
 'assiniboia',
 'havre',
 'sunnyside',
 'baraboo',
 'scottsbluff',
 'clarksburg',
 'glenwood springs',
 'nantucket',
 'nantucket',
 'alamosa',
 'virginia beach',
 'new bern',
 'alexander city',
 'ojinaga',
 'san felipe',
 'north battleford',
 'hanna',
 'revelstoke',
 'lillooet',
 'hauterive',
 'nelson',
 'sidney',
 'forestville',
 'redwater',
 'assiniboia',
 'superior',
 'saint-pacome',
 'cody',
 'bradford',
 'jac

In [13]:
# this is crazy that my new method results in fewer cities and more duplicates - how is that possible ?
# maybe I have to see the pins on the map to realize what happened- where is hamilton anyway and why so many?
# with lattice-y included we get seattle and tacoma but the list of cities is still shorter than I hoped
my_cities

['mandan',
 'smiths falls',
 'hamilton',
 'grandview',
 'baker city',
 'nampa',
 'maryville',
 'shelbyville',
 'madison heights',
 'eston',
 'stettler',
 'sioux lookout',
 'havre',
 'east wenatchee bench',
 'miles city',
 'le mars',
 'scottsbluff',
 'fallon',
 'clifton',
 'trinidad',
 'collierville',
 'havelock',
 'valdosta',
 'chapais',
 'ojinaga',
 'houma',
 'grande-riviere',
 'invermere',
 'cochrane',
 'dauphin',
 'dryden',
 'hudson bay',
 'kenora',
 'abbotsford',
 'hornepayne',
 'helena',
 'matane',
 'assiniboia',
 'sunnyside',
 'baraboo',
 'clarksburg',
 'glenwood springs',
 'nantucket',
 'alamosa',
 'virginia beach',
 'new bern',
 'alexander city',
 'san felipe',
 'north battleford',
 'hanna',
 'revelstoke',
 'lillooet',
 'hauterive',
 'nelson',
 'sidney',
 'forestville',
 'redwater',
 'superior',
 'saint-pacome',
 'cody',
 'bradford',
 'jacksonville',
 'oregon',
 'leesburg',
 'hastings',
 'brigantine',
 'salisbury',
 'malibu',
 'brownwood',
 'marsh harbour',
 'wynyard',
 'mataga

In [14]:
len(my_cities)

159

In [15]:
# finish set up for the map layer 
try_weights = [10,10,10,10,10,10,10,10,10,10]

# this should give me a list of all 10s the same lenth as the number of lats
my_weights = []
for lat in my_lats:
        my_weights.append(20)
           
# can I use 10 random points to make my own map layer ?
len(my_weights)

200

In [16]:
# only run this last cell when you are ready - these are the pin drops before we link them to a city
# download a copy of the last map before generating a new one - 20210728
start_lat = 39.03
start_lng = -95.72

# Only run a new map if you are sure you want to incur the cost and you must SAVE the previous map first
Run_this_code = False

if Run_this_code:
    fig = gmaps.figure(center=(start_lat, start_lng), zoom_level=4)

    heat_layer = gmaps.heatmap_layer(new_df, weights=my_weights[:maxsize])

    fig.add_layer(heat_layer)
    fig

# This is the map of just the pins we are dropping on the map, not the nearest city
# Do I get charged $$ every time I generate a new map ? ? ? ?

fig = gmaps.figure(center=(start_lat, start_lng), zoom_level=4)

heat_layer = gmaps.heatmap_layer(new_df, weights=my_weights[:maxsize])

fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))